In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/revpromptlist/Revised_prompt_list.csv
/kaggle/input/finalsurpriseme-bigtrain/BigtrainandSurpriseMe.csv
/kaggle/input/finalsurpriseme-bigtrain/PromptTest.csv
/kaggle/input/finalsurpriseme-bigtrain/SurpriseMeTest.csv
/kaggle/input/fixedoneword/FinalSurpriseMeOneWordX2.csv
/kaggle/input/orignaltraintest/PromptTrain.csv
/kaggle/input/orignaltraintest/PromptTest.csv
/kaggle/input/bigtrainandoneword/PromptTest.csv
/kaggle/input/bigtrainandoneword/BigTrainingOneWord.csv
/kaggle/input/shortenedrevlist/ShortenedRevList.csv
/kaggle/input/onewordtrain/PromptTest.csv
/kaggle/input/onewordtrain/TrainingWtihOneWord.csv
/kaggle/input/fiexpt2/FinalSurpriseMeOneWordX2.csv


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 792.8 kB/s eta 0:00:000:00:01


In [3]:
import pandas as pd
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import accelerate
import joblib

2024-08-08 08:30:45.308067: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 08:30:45.308192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 08:30:45.592815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
label2id = {"Arts_and_crafts": 0, 
            "Education": 1,
            "Food_and_drink": 2, 
            "Fashion_and_apparel": 3,
            "Jewellery_and_accessories": 4, 
            "Health_and_Beauty": 5,
            "Technology_and_computing": 6, 
            "Toys": 7,
            "Electronics_and_gadgets": 8,
            "Events_and_Entertainment": 9, 
            "Personal_Resume_CV": 10,
            "Portfolio": 11, 
            "Real_estate_and_interiors": 12,
            "Restaurants": 13, 
            "Sports_and_Outdoors": 14,
            "Travel_and_Tourism": 15, 
            "Automotive": 16,
            "Insurance": 17,
            "Photography": 18, 
            "Professional_services":19}

In [5]:
list1 = label2id.keys()
list1

dict_keys(['Arts_and_crafts', 'Education', 'Food_and_drink', 'Fashion_and_apparel', 'Jewellery_and_accessories', 'Health_and_Beauty', 'Technology_and_computing', 'Toys', 'Electronics_and_gadgets', 'Events_and_Entertainment', 'Personal_Resume_CV', 'Portfolio', 'Real_estate_and_interiors', 'Restaurants', 'Sports_and_Outdoors', 'Travel_and_Tourism', 'Automotive', 'Insurance', 'Photography', 'Professional_services'])

In [6]:
id2label = {0: 'Arts_and_crafts', 1: 'Education', 2: 'Food_and_drink', 3: 'Fashion_and_apparel', 4: 'Jewellery_and_accessories', 5: 'Health_and_Beauty', 6: 'Technology_and_computing', 7: 'Toys', 8: 'Electronics_and_gadgets', 9: 'Events_and_Entertainment', 10: 'Personal_Resume_CV', 11: 'Portfolio', 12: 'Real_estate_and_interiors', 13: 'Restaurants', 14: 'Sports_and_Outdoors', 15: 'Travel_and_Tourism', 16: 'Automotive', 17: 'Insurance', 18: 'Photography', 19: 'Professional_services'}

In [7]:
Train_file_path = r"/kaggle/input/fiexpt2/FinalSurpriseMeOneWordX2.csv"
OneWordTrain_Path = r"/kaggle/input/onewordtrain/TrainingWtihOneWord.csv"
Test_file_path = r"/kaggle/input/finalsurpriseme-bigtrain/PromptTest.csv"

In [8]:
from datasets import load_dataset

In [9]:
dataset = load_dataset('csv', data_files={'train': Train_file_path, 'test': Test_file_path})
train_data = dataset['train']
test_data = dataset['test']
shuffled_train_data = train_data.shuffle(seed=42)
shuffled_test_data = test_data.shuffle(seed=42)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
shuffled_train_data_encoded = shuffled_train_data.class_encode_column("labels")
SDE_train_aligned = shuffled_train_data_encoded.align_labels_with_mapping(label2id, "labels")
shuffled_test_data_encoded = shuffled_test_data.class_encode_column("labels")
SDE_test_aligned = shuffled_test_data_encoded.align_labels_with_mapping(label2id, "labels")

Casting to class labels:   0%|          | 0/14106 [00:00<?, ? examples/s]

Aligning the labels:   0%|          | 0/14106 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/800 [00:00<?, ? examples/s]

Aligning the labels:   0%|          | 0/800 [00:00<?, ? examples/s]

In [11]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=False)
tokenized_SDE_train_aligned = SDE_train_aligned.map(preprocess_function, batched=True)
tokenized_SDE_test_aligned = SDE_test_aligned.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Map:   0%|          | 0/14106 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [12]:
PreTrainmodel = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=20, id2label=id2label, label2id=label2id,ignore_mismatched_sizes=True)

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly 

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=12,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-6,
    warmup_ratio=0.25,
    weight_decay=0.2,
    logging_dir="path/to/logging/directory",
    logging_steps=100,
    evaluation_strategy="epoch",
    eval_steps=None,
    save_strategy="epoch",
    save_total_limit=2,
    save_steps=None,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    gradient_accumulation_steps=1,
    seed=42,
    report_to="none"
)

trainer = Trainer(
    model=PreTrainmodel,
    args=training_args,
    train_dataset=tokenized_SDE_train_aligned,
    eval_dataset=tokenized_SDE_test_aligned,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PreTrainmodel.save_pretrained(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,2.911000,2.722435,0.485000
2,2.189100,1.530563,0.943750
3,1.185300,0.452903,0.990000
4,0.625600,0.157803,0.996250
5,0.432000,0.083743,0.997500
6,0.321400,0.054642,0.996250
7,0.249800,0.037276,0.996250
8,0.192400,0.030355,0.996250
9,0.168300,0.024778,0.996250
10,0.164100,0.023292,0.996250


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/merges.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [14]:
my_model_path = r"/kaggle/working/results/checkpoint-10584"

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(my_model_path)
tokenizer = AutoTokenizer.from_pretrained(my_model_path)

In [19]:
seq = "For my fine dining restaurant"


In [20]:
inputs = tokenizer(seq, return_tensors="pt")
with torch.no_grad():
    
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'Restaurants'

In [21]:
Revised_prompt_list_path= r"/kaggle/input/finalsurpriseme-bigtrain/SurpriseMeTest.csv"

In [22]:
dataset2 = load_dataset('csv', data_files={'train': Revised_prompt_list_path, 'test': Revised_prompt_list_path})
Rev_train_data = dataset2['train']
Rev_test_data = dataset2['test']

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [23]:
labels_list = Rev_test_data['labels']
list_prompts = Rev_test_data['text']

In [25]:
labels_list

['Real_estate_and_interiors',
 'Professional_services',
 'Real_estate_and_interiors',
 'Professional_services',
 'Jewellery_and_accessories',
 'Insurance',
 'Sports_and_Outdoors',
 'Real_estate_and_interiors',
 'Sports_and_Outdoors',
 'Technology_and_computing',
 'Sports_and_Outdoors',
 'Fashion_and_apparel',
 'Restaurants',
 'Fashion_and_apparel',
 'Professional_services',
 'Restaurants',
 'Arts_and_crafts',
 'Toys',
 'Real_estate_and_interiors',
 'Fashion_and_apparel',
 'Professional_services',
 'Professional_services',
 'Sports_and_Outdoors',
 'Events_and_Entertainment',
 'Food_and_drink',
 'Jewellery_and_accessories',
 'Portfolio',
 'Real_estate_and_interiors',
 'Professional_services',
 'Technology_and_computing',
 'Real_estate_and_interiors',
 'Insurance',
 'Food_and_drink',
 'Health_and_Beauty',
 'Arts_and_crafts',
 'Professional_services',
 'Events_and_Entertainment',
 'Arts_and_crafts',
 'Arts_and_crafts',
 'Sports_and_Outdoors',
 'Travel_and_Tourism',
 'Fashion_and_apparel',


In [ ]:
print(os.listdir('.'))

In [26]:
EmptyPredictionCheckerTest = []
PromptCounter = 0
for text in list_prompts:
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        EmptyPredictionCheckerTest.append(model.config.id2label[predicted_class_id])
        PromptCounter += 1 
        print(PromptCounter,"of", len(list_prompts), "prompts completed:")
        print("for the prompt:", text, "the predicted industry was", model.config.id2label[predicted_class_id] )

1 of 108 prompts completed:
for the prompt: for my real estate agency called Dream Homes, make pastel sage green as primary color. the predicted industry was Real_estate_and_interiors
2 of 108 prompts completed:
for the prompt: for my resume service called Perfect Profile, make Misty Rose as primary color. the predicted industry was Personal_Resume_CV
3 of 108 prompts completed:
for the prompt: for my real estate and interiors business called DreamSpace Designs, make Misty Rose as primary color. the predicted industry was Real_estate_and_interiors
4 of 108 prompts completed:
for the prompt: for my virtual assistant service called Digital Helping Hand, make Light Sky Blue as primary color. the predicted industry was Professional_services
5 of 108 prompts completed:
for the prompt: for my jewelry boutique called Elegant Sparkle, make pastel lavender as primary color. the predicted industry was Jewellery_and_accessories
6 of 108 prompts completed:
for the prompt: for my insurance agency c

In [27]:
counter = 0
emptylistguesses = []
emptylistactuals = []
for i in range(len(EmptyPredictionCheckerTest)):
  if EmptyPredictionCheckerTest[i] != labels_list[i]:
    emptylistguesses.append(EmptyPredictionCheckerTest[i])
    emptylistactuals.append(labels_list[i])
    counter += 1

In [28]:
print("percentage accuracy for your model with suprise me labelled data:", ((len(EmptyPredictionCheckerTest) - counter) / len(EmptyPredictionCheckerTest)) * 100,"%" )

percentage accuracy for your model with suprise me labelled data: 94.44444444444444 %


In [29]:
def combine_lists(a, b):
    combined_list = []
    for item_a, item_b in zip(a, b):
        combined_list.append(item_a + ' Actual:' + item_b)
    return combined_list

In [30]:
actuals_list = combine_lists(emptylistguesses,emptylistactuals)

In [31]:
actuals_list

['Personal_Resume_CV Actual:Professional_services',
 'Professional_services Actual:Automotive',
 'Fashion_and_apparel Actual:Jewellery_and_accessories',
 'Personal_Resume_CV Actual:Professional_services',
 'Sports_and_Outdoors Actual:Health_and_Beauty',
 'Sports_and_Outdoors Actual:Health_and_Beauty']